In [1]:
import pandas as pd
#from collections import defaultdict
#import csv
import re 
from string import punctuation
from nltk import word_tokenize, wordpunct_tokenize
from collections import Counter
import main
from importlib import reload
import numpy as np

In [2]:
filename="RS_2011-01_1000"
outputname='filtered_'+filename[3:10]
sub_list= set(['reddit.com','leagueoflegends', 'gaming', 'DestinyTheGame', 'DotA2', 'ContestofChampions', 'StarWarsBattlefront', 'Overwatch', 'WWII', 'hearthstone', 'wow', 'heroesofthestorm', 'destiny2', 'darksouls3', 'fallout', 'SuicideWatch', 'depression', 'OCD', 'dpdr', 'proED', 'Anxiety', 'BPD', 'socialanxiety', 'mentalhealth', 'ADHD', 'bipolar', 'buildapc', 'techsupport', 'buildapcforme', 'hacker', 'SuggestALaptop', 'hardwareswap', 'laptops', 'computers', 'pcmasterrace', 'relationshps', 'relationship_advice', 'breakups', 'dating_advice', 'LongDistance', 'polyamory', 'wemetonline', 'MDMA', 'Drugs', 'trees', 'opiates', 'LSD', 'tifu', 'r4r', 'AskReddit', 'reddit.com', 'tipofmytongue', 'Life', 'Advice', 'jobs', 'teenagers', 'HomeImprovement', 'redditinreddit', 'FIFA', 'nba', 'hockey', 'nfl', 'mls', 'baseball', 'BokuNoHeroAcademia', 'anime', 'movies', 'StrangerThings'])
minwords=100

In [3]:
alldata = pd.read_json(filename, lines=True)
if sub_list!=set():
    alldata=alldata[alldata.subreddit.isin(sub_list)]


In [4]:
alldata

,approved_by,author,author_flair_css_class,author_flair_text,banned_by,clicked,created,created_utc,distinguished,domain,...,saved,score,selftext,selftext_html,subreddit,subreddit_id,thumbnail,title,ups,url
9,NaN,thenewguy729,None,None,NaN,False,1293944352,1293944352,None,self.gaming,...,False,1,So I bought Amnesia on the Steam sale the othe...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",gaming,t5_2qh03,self,Can You Run It lied to me. Help?,1,http://www.reddit.com/r/gaming/comments/eut3s/...
12,NaN,frnak,None,None,NaN,False,1293944322,1293944322,None,self.AskReddit,...,False,2,"People just hang up, like both parties know ex...","&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",AskReddit,t5_2qh1i,self,Why can't phone calls on TV and in movies end ...,4,http://www.reddit.com/r/AskReddit/comments/eut...
18,NaN,SteveMarcus,None,None,NaN,False,1293944292,1293944292,None,self.AskReddit,...,False,2,So I deliver(ed?) pizza for a local establishm...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",AskReddit,t5_2qh1i,self,I was impersonally fired from my pizza deliver...,2,http://www.reddit.com/r/AskReddit/comments/eut...
19,NaN,[deleted],None,None,NaN,False,1293944269,1293944269,None,imgur.com,...,False,3,,None,trees,t5_2r9vp,default,I am at [5] and I just noticed I was eating...,8,http://imgur.com/l0pjn
20,NaN,ooglytoop7272,None,None,NaN,False,1293944267,1293944267,None,self.AskReddit,...,False,0,They've been saying the script is half done fo...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",AskReddit,t5_2qh1i,self,Does anybody know the status on the Arrested D...,5,http://www.reddit.com/r/AskReddit/comments/eut...
22,NaN,slh1191,None,None,NaN,False,1293944264,1293944264,None,hollyhotty.com,...,False,0,,None,reddit.com,t5_6,default,kelly brooks,0,http://www.hollyhotty.com/?p=1837
24,NaN,solton,None,None,NaN,False,1293944261,1293944261,None,themallblog.com,...,False,1,,None,reddit.com,t5_6,default,PLR Profits,1,http://www.themallblog.com/profit-plr-0
28,NaN,[deleted],None,None,NaN,False,1293944254,1293944254,None,gaming.stackexchange.com,...,False,1,,None,gaming,t5_2qh03,default,Xbox Kinect user is experiencing vision proble...,1,http://gaming.stackexchange.com/questions/1371...
34,NaN,kcocampo,None,None,NaN,False,1293944209,1293944209,None,self.reddit.com,...,False,1,,None,reddit.com,t5_6,default,"Once Learned, Video Editing Can Be Easy",1,http://www.reddit.com/r/reddit.com/comments/eu...
35,NaN,[deleted],None,None,NaN,False,1293944208,1293944208,None,youtubedoubler.com,...,False,1,,None,reddit.com,t5_6,default,"Dance with Manbungo ,The African Warrior,every...",1,http://youtubedoubler.com/?video1=s-MqaCCbDco


In [5]:
selftext= alldata[alldata['is_self']] 	\
    .loc[:,['id','subreddit','selftext']]
selftext


,id,subreddit,selftext
9,eut3s,gaming,So I bought Amnesia on the Steam sale the othe...
12,eut3p,AskReddit,"People just hang up, like both parties know ex..."
18,eut3i,AskReddit,So I deliver(ed?) pizza for a local establishm...
20,eut3g,AskReddit,They've been saying the script is half done fo...
34,eut30,reddit.com,
36,eut2y,gaming,Or does anyone know where someone has analyzed...
37,eut2x,reddit.com,
42,eut2s,AskReddit,I'd rather not download a torrent if at all po...
53,eut2h,AskReddit,I could've never beaten Majora's Mask without ...
55,eut2f,reddit.com,


In [6]:
selftext['selftext']=selftext.transform({'selftext': lambda x: main.tokenize(x)})#, lambda x: tokenize(x)[1]] })

selftext[['counter', 'wordcount']] = selftext['selftext'].apply(pd.Series)



In [7]:
filtered= selftext[selftext['wordcount'] >=minwords] \
        .loc[:,['id','subreddit','counter', 'wordcount']]


In [8]:
filtered

,id,subreddit,counter,wordcount
9,eut3s,gaming,"{'so': 1, 'i': 9, 'bought': 3, 'amnesia': 3, '...",182
18,eut3i,AskReddit,"{'so': 1, 'i': 10, 'deliver': 1, '(': 5, 'ed':...",241
57,eut2d,AskReddit,"{'okay': 1, 'so': 2, 'my': 2, 'cousins': 1, 'a...",152
86,eut1k,AskReddit,"{'google': 1, 'queries': 1, 'aside': 1, ',': 5...",104
186,eusyq,AskReddit,"{'so': 5, 'my': 15, 'sister': 10, '(': 16, 'i'...",624
190,eusym,reddit.com,"{'katey': 4, 'says': 7, ':': 7, 'i': 5, 'bough...",105
201,eusyb,AskReddit,"{'my': 1, 'wife': 1, 'is': 3, 'a': 3, 'talente...",127
223,eusxn,AskReddit,"{'i': 2, 'am': 1, 'trying': 2, 'to': 6, 'set':...",114
230,eusxf,gaming,"{'i': 5, ''m': 2, 'trying': 2, 'to': 4, 'get':...",145
265,euswf,reddit.com,"{'welcome': 1, 'to': 4, 'www.valentinediamondc...",146


# Dictionaries

In [9]:
reload(main)
main.getdicts("LIWC2007_updated.dic")
absolutist = ['absolutely', 'all', 'always', 'complete', 'competely','constant', 'constantly', 'definitely', 'entire', 'ever', 'every', 'everyone', 'everything', 'full', 'must', 'never', 'nothing', 'totally','whole']
main.WordCollection(0,'absolutist', absolutist)


In [10]:
abscounts = main.calculatePosts2(filtered)
abscounts

,wordcount,funct,pronoun,ppron,i,we,you,shehe,they,ipron,...,achieve,leisure,home,money,relig,death,assent,nonfl,filler,absolutist
subreddit,,,,,,,,,,,,,,,,,,,,,
AskReddit,8002,0.514621,0.153462,0.106473,0.070607,0.005624,0.004499,0.020745,0.004999,0.044364,...,0.008748,0.008248,0.003374,0.003749,0.001250,0.000125,0.001500,0.000750,0.002624,0.008373
SuicideWatch,329,0.513678,0.130699,0.075988,0.072948,0.000000,0.000000,0.003040,0.000000,0.054711,...,0.009119,0.000000,0.000000,0.003040,0.006079,0.003040,0.000000,0.000000,0.000000,0.036474
depression,1637,0.549175,0.191814,0.127062,0.084301,0.000000,0.007330,0.032376,0.003054,0.059866,...,0.022602,0.004887,0.001833,0.003054,0.001833,0.001222,0.000611,0.001222,0.004887,0.017104
gaming,775,0.443871,0.127742,0.078710,0.047742,0.000000,0.015484,0.009032,0.006452,0.047742,...,0.018065,0.014194,0.000000,0.018065,0.000000,0.000000,0.002581,0.000000,0.001290,0.012903
hockey,193,0.507772,0.103627,0.046632,0.031088,0.000000,0.005181,0.000000,0.010363,0.056995,...,0.010363,0.031088,0.005181,0.000000,0.005181,0.000000,0.000000,0.005181,0.000000,0.010363
movies,100,0.460000,0.130000,0.100000,0.030000,0.000000,0.000000,0.070000,0.000000,0.030000,...,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
reddit.com,1021,0.434868,0.124388,0.089128,0.061704,0.007835,0.005877,0.012733,0.000979,0.032321,...,0.011753,0.017630,0.006856,0.008815,0.000000,0.000000,0.002938,0.001959,0.002938,0.001959
relationship_advice,928,0.587284,0.169181,0.119612,0.060345,0.015086,0.000000,0.039871,0.004310,0.046336,...,0.010776,0.004310,0.000000,0.003233,0.000000,0.000000,0.000000,0.004310,0.003233,0.012931
techsupport,463,0.360691,0.112311,0.060475,0.056156,0.000000,0.002160,0.000000,0.002160,0.051836,...,0.008639,0.004320,0.008639,0.004320,0.000000,0.000000,0.002160,0.000000,0.000000,0.012959
